In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q transformers datasets sentencepiece

!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate

!pip install -q rouge-score

!pip install -q torchmetrics

!pip install -q git+https://github.com/huggingface/accelerate

!git clone -q https://github.com/huggingface/transformers.git
!cd transformers
!pip install  -q -e.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 31.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:

from transformers import MBart50TokenizerFast
from transformers import MBartForConditionalGeneration

# model_name_or_path = 'facebook/mbart-large-50'
# model_name_or_path = '/content/drive/MyDrive/mBART_QA/checkpoint-13234'
model_name_or_path = '/content/drive/MyDrive/mBART_QA/checkpoint-4409'

tokenizer = MBart50TokenizerFast.from_pretrained(model_name_or_path)
model = MBartForConditionalGeneration.from_pretrained(model_name_or_path)

comment the following lines if you want to fine-tune on different datasets

In [ ]:
# from datasets import concatenate_datasets, load_dataset

# train_data = load_dataset("Gholamreza/pquad", split="train")
# valid_data = load_dataset("Gholamreza/pquad", split="validation")
# test_data = load_dataset("Gholamreza/pquad", split="test")

# concat_dataset = concatenate_datasets([train_data, valid_data, test_data])
# dataset=concat_dataset

In [ ]:
from datasets import Dataset
import random
import json

# data_files = ['total_augmented.json', 'train.json', 'dev.json']

data_files = ['train_v5.json', 'back_translated_v5.json', 'extra_translations_v5.json', 'dev_v5.json']

total_data_list = []

for file in data_files:
  path = '/content/drive/MyDrive/' + file
  with open(path, 'r', encoding = 'utf-8-sig') as f:
    content = json.load(f)
    total_data_list = total_data_list + content

random.shuffle(total_data_list)

dataset = Dataset.from_list(total_data_list)

In [ ]:
dataset

Dataset({
    features: ['context', 'pq_id', 'question', 'translation_file', 'answers'],
    num_rows: 19596
})

In [ ]:
# del concat_dataset, train_data, valid_data

small_dataset = dataset

# small_dataset= dataset.filter(lambda example: len(example['answers']['text'])>0)
# small_dataset= small_dataset.filter(lambda example: 'چرا' not in example['context'])
split_data = small_dataset.train_test_split(test_size=0.1, seed= 42, shuffle=True)

dataset = split_data

del split_data

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'pq_id', 'question', 'translation_file', 'answers'],
        num_rows: 17636
    })
    test: Dataset({
        features: ['context', 'pq_id', 'question', 'translation_file', 'answers'],
        num_rows: 1960
    })
})

In [ ]:
# def format_dataset(example):
#   return {'input': 'question: ' + example['question'] + ' context: ' + example['context'], 'target': example['answers'][0]['text']}

def format_dataset(example):
  context = example['context']
  question = example['question']
  # answer = example['answers']['text'][0]
  answer = example['answers'][0]['text']
  question_plus = f"question: {str(question)}"
  question_plus += f" context: {str(context)} </s>"

  answer_plus = answer
  answer_plus = f"{answer_plus} </s>"

  # return {'input': 'question: ' + example['question'] + ' context: ' + example['context'], 'target': example['answers'][0]['text']}
  return {'input': question_plus, 'target': answer_plus}

dataset = dataset.map(format_dataset, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/17636 [00:00<?, ? examples/s]

Map:   0%|          | 0/1960 [00:00<?, ? examples/s]

In [ ]:
test_data = dataset['test']

test_data[0]

# train_ds=  dataset['train'].map(encode)
# valid_ds=  dataset['test'].map(encode)


{'input': 'question: خداوند آفریننده چه چیزهایی است؟ context: پروردگارا به من بهره\u200cای از فرمانروایی بخشیدی و به من بهره\u200cای از تعبیر خواب آموختی، ای پدید آورنده آسمانها و زمین، تو در دنیا و آخرت سرور منی، مرا مسلمان بمیران و به نیکان باز رسان\u200c </s>',
 'target': 'آسمانها و زمین </s>'}

In [ ]:
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], truncation=True, padding='max_length', max_length=564)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], truncation=True, padding='max_length', max_length=64)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [ ]:
dataset = dataset.map(convert_to_features, batched=True, remove_columns=dataset['train'].column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

# train_ds.set_format(type='torch', columns=columns)
# valid_ds.set_format(type='torch', columns=columns)

dataset.set_format(type='torch', columns=columns)

Map:   0%|          | 0/17636 [00:00<?, ? examples/s]

Map:   0%|          | 0/1960 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer
# data_collator = default_data_collator
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

# set training arguments - Feel free to adapt it
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/mBART_QA",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    save_strategy="epoch"
    # save_strategy="no",
    #eval_steps=1000,
    # push_to_hub=False
    #fp16=True,
)


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    # data_collator=T2TDataCollator(),
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.005200,0.030895


Checkpoint destination directory /content/drive/MyDrive/mBART_QA/checkpoint-4409 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


TrainOutput(global_step=4409, training_loss=0.00816981798939003, metrics={'train_runtime': 5364.5055, 'train_samples_per_second': 3.288, 'train_steps_per_second': 0.822, 'total_flos': 2.1050584307859456e+16, 'train_loss': 0.00816981798939003, 'epoch': 1.0})

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# prompt = "question: خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟ context: اي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم، و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"

context = "ي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم، و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"

question = "خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟"

# context = """We went on a trip to Europe. We had our breakfast at 7 am in the morning at \
# the nearby coffee shop. Wore a dark blue over coat for our first visit to Louvre Museum \
# to experience history and art."""

# question = "At what time did we had breakfast?"

prompt = f"question: {question} context: {context} </s>"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids,
                                    num_beams=7,
                                    num_return_sequences=7,
                                    top_k=50,
#                                     top_p=.2,
                                    temperature=0.95,

                                    max_length=200)
for generated in generated_ids:
    generated_text = tokenizer.decode(generated,skip_special_tokens=True)
print(generated_text)



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


گونه


In [ ]:
context = """ي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم،  \
و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"""

question = "خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟"
print(context)
print(question)


# context = """We went on a trip to Europe. We had our breakfast at 7 am in the morning at \
# the nearby coffee shop. Wore a dark blue over coat for our first visit to Louvre Museum \
# to experience history and art."""

# question = "At what time did we had breakfast?"
# print(context)
# print(question)

ي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم،  و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د
خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_text =  f"answer: {question} context: {context} </s>"
input_ids = tokenizer(input_text,
                         return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)
# input_ids = encoded_query["input_ids"]
# attention_mask = input_ids["attention_mask"]
attention_mask = tokenizer(input_text,
                         return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

generated_answer = model.generate(input_ids, attention_mask = attention_mask,
                                 max_length=64, top_p=0.95, top_k=50)
for each in generated_answer:
  decoded_answer = tokenizer.decode(each, skip_special_tokens=True)
print("Answer: ", decoded_answer)

Answer:  طور


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


# evaluation

In [ ]:
import json
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('/content/drive/MyDrive/dev.json', 'r') as f:
  dev_data_list = json.load(f)

predicted_output = []
ground_truth = []

for qa in dev_data_list[:20]:
  ground_truth.append(qa['answers'][0]['text'])

  input_text =  f"answer: {qa['question']} context: {qa['context']} </s>"
  input_ids = tokenizer(input_text,
                          return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)

  attention_mask = tokenizer(input_text,
                          return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

  generated_answer = model.generate(input_ids, attention_mask = attention_mask,
                                  max_length=64, top_p=0.95, top_k=50)
  for each in generated_answer:
    decoded_answer = tokenizer.decode(each, skip_special_tokens=True)

  predicted_output.append(decoded_answer)


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

f1 = f1_score(ground_truth, predicted_output, average='micro')

f1

0.3

In [ ]:
def compute_exact_match_score(pred, gt):
    num_correct = 0

    for p, g in zip(pred, gt):
        if p == g:
            num_correct += 1

    exact_match_score = num_correct / len(pred)
    return exact_match_score

exact_match_score = compute_exact_match_score(predicted_output, ground_truth)
print("Exact match score: {:.2f}".format(exact_match_score))


Exact match score: 0.30


In [ ]:
ground_truth

['روح',
 'موسی',
 'اسرافکاران',
 'تندبادی است که عذاب دردناکی در آن است',
 'شراب طهوری آمیخته با زنجبیل',
 'نوزده نفر',
 'راه\u200cرفتن',
 'فضل خود',
 'تقوای الهی',
 'غیر خدا',
 'رسالتهای الهی',
 'ظالمان',
 'القاب زشت و ناپسند',
 'وحی',
 'صد تازیانه',
 'شدّت و رنجی که به او رسیده',
 'یک',
 'کسانی که به خدا دروغ می\u200cبندند',
 'زراعتی',
 'آنچه انجام می\u200cدادند']

In [ ]:
predicted_output

['روح',
 'موسی',
 'انسان',
 'احقاف',
 'حریر',
 'نوزده',
 'فرزندش',
 'فضل خود',
 'تقوای',
 'پیامبر',
 'ایمانی',
 'تمام کسانی را که این قرآن بر من وحی شده، تا شما و تمام کسانی را که این قرآن بر من وحی شده، تا شما و تمام کسانی را که این',
 'ایمان',
 'وحی',
 'یک',
 'شدّت و رنج',
 'یک',
 'خدا',
 'زراعتی',
 'فرشتگان']

In [ ]:
for a in dev_data_list[:20]:
  print(a['question'])

فرشتگان ذکر شده در قرآن چه کسانی هستند؟
کدام پیامبر از سوی فرعونیان تکذیب شد؟
اعمال چه کسانی برایشان زینت داده شده است؟
قوم عاد به چه بلایی گرفتار شدند؟
جامهای بهشتی لبریز از چه چیزی است؟
چند نفر از فرشتگان نگهبان دوزخ هستند؟
لقمان از پسرش خواست در چه چیزی اعتدال را رعایت کند؟
خداوند مومنان تنگدست را از چه چیز بی‌نیاز می‌سازد؟
مومنان باید چه چیزی را در امور خود پیشه کنند؟
اگر قرآن از سوی چه کسی بود، اختلاف فراوانی در آن می‌یافتند؟
پیامبران پیشین چه چیز را بین مردم تبلیع می‌کردند؟
چه کسانی رستگار نخواهند شد؟
در قرآن سفارش شده است که مردم با چه چیزی یکدیگر را یاد نکنند؟
پیامبر تنها به چه وسیله‌ای مردم را انذار می‌کند؟
هر یک از زن و مرد زناکار را باید چند تازیانه بزنند؟
اگر بعد از چه چیز به انسان نعمت چشانده شود غرق غفلت می‌شود؟
خدایی که بنده‌اش را در چند شب، از مسجد الحرام به مسجد الاقصی برد؟
چه کسانی رستگار نخواهند شد؟
خداوند با آب از زمین چه چیز را خارج می‌سازد که رنگهای مختلف دارد؟
‌خداوند در آخرت انسانها را از چه با خبر می‌کند؟
